Michał Chojnacki 151859

Jakub Głowacki 151865

Informatyka

# Wykrywanie gry w bilard

## Zbiór danych

Analizowaliśmy fragment gry w bilard zawarty w filmie na platformie youtube wykonany przez użytkownika
The Billiard Room by Jason Creglia: https://www.youtube.com/watch?v=_H7ipblfNzs

Plik wideo został pobrany do pliku mp4.

## Wyniki

Film wynikowy został zapisany również w formacie mp4, natomiast każde zdarzenie powodowało wypisanie obecnego stanu gry.

## Kod programu

Nasz program wykrywa położenie kul a następnie na ich podstawie wykrywa zdarzenie wbicia bili do łuzy.

### Skrypt działa w następujący sposób:

Inicjalizacja – otwiera plik wideo, odczytuje pierwszą klatkę i na jej podstawie wycina małe ROI wokół każdej bili, tworząc z nich szablony wraz z maskami kołowymi.

Śledzenie – dla każdej klatki przelicza dopasowanie szablonu lokalnie wokół ostatniej pozycji (z fallbackem na całą klatkę) i wybiera najlepsze detekcje, eliminując nakładające się wyniki.

Rysowanie – nanosi na każdą kulkę okrąg oraz wartość dopasowania

Logika „throw out” – monitoruje, ile kolejnych sekund dana kulka ma dopasowanie poniżej progu, jeśli dluzlona wartość to zakładamy ze bila została wbita

Eksport – zapisuje wynikowy film i wypisuje w konsoli momenty wybicia każdej bili.


In [ ]:
import cv2
import numpy as np
from IPython.display import FileLink

VIDEO_IN             = 'Vid1J.mp4'
VIDEO_OUT            = 'tracked_10balls_throwout.mp4'
MAX_SECS             = 90
DETECT_THRESHOLD     = 0.7    # minimalna wartość dopasowania do uznania za wykrycie
THROWOUT_THRESHOLD   = 0.85   # jeśli 3 razy poniżej tej wartości → throw out
WIN_MARGIN           = 50     # px wokół ostatniej pozycji do lokalnego searchu
DIST_SUPPRESS_FACTOR = 1      # suppression distance factor
THROWOUT_SEC_COUNT   = 4      # ile sekund poniżej THROWOUT_THRESHOLD z rzędu → throw out

cap = cv2.VideoCapture(VIDEO_IN)
assert cap.isOpened(), "Nie udało się otworzyć wideo!"
fps  = cap.get(cv2.CAP_PROP_FPS)
w    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h    = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
ret, first = cap.read()
assert ret, "Nie udało się wczytać pierwszej klatki!"

rois = [
    (210,230,40,40),  # 1 white
    (190,350,40,40),  # 2 yellow full
    ( 90,345,40,40),  # 3 yellow half
    (338,345,40,40),  # 4 red
    (480,200,40,40),  # 5 dark red
    (762,198,40,40),  # 6 green
    (485,490,40,40),  # 7 orange
    (767,490,40,40),  # 8 black
    (915,340,40,40),  # 9 blue
    (1060,337,40,40), #10 purple
]

N = len(rois)
templates, masks, radii = [], [], []
last_pos, active = [], []
low_count = [0]*N

for x,y,tw,th in rois:
    patch = first[y:y+th, x:x+tw]
    tmpl  = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
    m     = np.zeros((th,tw), np.uint8)
    cv2.circle(m, (tw//2,th//2), min(tw,th)//2, 255, -1)
    templates.append(tmpl)
    masks.append(m)
    r = min(tw,th)//2
    radii.append(r)
    last_pos.append((x+tw//2, y+th//2))
    active.append(True)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out    = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w,h))
colors = [tuple(np.random.randint(0,255,3).tolist()) for _ in range(N)]

last_drawn_val = [None]*N

scored_times = []
pocket_count = 0
total_frames = int(MAX_SECS * fps)
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

for idx in range(total_frames):
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if idx % int(fps) == 0:
        last_drawn_val = [None]*N

    candidates = []
    for i in range(N):
        if not active[i]:
            continue
        tmpl = templates[i]
        tw, th = tmpl.shape[1], tmpl.shape[0]
        cx, cy = last_pos[i]

        x0 = max(0, cx - WIN_MARGIN - tw//2)
        y0 = max(0, cy - WIN_MARGIN - th//2)
        x1 = min(w - tw, cx + WIN_MARGIN - tw//2)
        y1 = min(h - th, cy + WIN_MARGIN - th//2)
        roi = gray[y0:y1+th, x0:x1+tw]

        res = cv2.matchTemplate(roi, tmpl, cv2.TM_CCOEFF_NORMED, mask=masks[i])
        _, val, _, loc = cv2.minMaxLoc(res)
        if val < DETECT_THRESHOLD:
            res_full = cv2.matchTemplate(gray, tmpl, cv2.TM_CCOEFF_NORMED, mask=masks[i])
            _, val, _, loc = cv2.minMaxLoc(res_full)
            if val < DETECT_THRESHOLD:
                continue
            else:
                x_match, y_match = loc
        else:
            x_match, y_match = loc
            x_match += x0
            y_match += y0

        cx_new = x_match + tw//2
        cy_new = y_match + th//2
        candidates.append((i, val, cx_new, cy_new, radii[i]))

    candidates.sort(key=lambda c: c[1], reverse=True)
    picked = []
    for i,val,cx,cy,r in candidates:
        if any(np.hypot(cx-px, cy-py) < r*DIST_SUPPRESS_FACTOR for (_,_,px,py,_) in picked):
            continue
        picked.append((i,val,cx,cy,r))

    for i,val,cx,cy,r in picked:
        last_pos[i]       = (cx, cy)
        last_drawn_val[i] = val
        cv2.circle(frame, (int(cx),int(cy)), r, colors[i], 3)
        cv2.putText(frame, f"{val:.2f}",
                    (int(cx-r), int(cy-r-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[i], 1)

    for i in range(N):
        if active[i] and last_drawn_val[i] is not None and not any(i==p[0] for p in picked):
            cx, cy = last_pos[i]
            r      = radii[i]
            cv2.circle(frame, (int(cx),int(cy)), r, colors[i], 1)

    out.write(frame)

    if (idx+1) % int(fps) == 0:
        sec = (idx+1)//int(fps)
        print(f"--- {sec}s drawn match values:")
        for i, v in enumerate(last_drawn_val):
            label = f"{v:.2f}" if v is not None else "-"
            print(f" Ball {i+1}: {label}", end=" |")
            if active[i]:
                if v is None or v < THROWOUT_THRESHOLD:
                    low_count[i] += 1
                else:
                    low_count[i] = 0
                if low_count[i] >= THROWOUT_SEC_COUNT:
                    active[i]    = False
                    pocket_count += 1
                    scored_times.append((i, sec))
                    print(f"\n Ball {i+1} thrown out at {sec}s ▶ total {pocket_count}", end="")
        print(f"   pocketed: {pocket_count}\n")

cap.release()
out.release()

print("Finished.")
print("Pocketed balls:")
for i, sec in scored_times:
    print(f" • Ball {i+1} at {sec}s")

print("Saved →", VIDEO_OUT)
display(FileLink(VIDEO_OUT))
